# Investigating Ca II lines

The `RH` code is already run, and all outputs are included in the folder. Following are analyses from the output of the code, and recipes of how I ran the code. 

First, we run the `RH` code as stated [here](https://tiagopereira.space/ast5210/CaII_formation/).

Modify `atmos.input` to include all 10 atoms, and in `keyword.input` set `15D_WRITE_POPS = TRUE`.

... relevant imports.

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d
from helita.vis import rh15d_vis
from IPython.display import HTML, display

After loading the output into data, we can find the indices of 392.8 < $\lambda$ < 394.0 (the Ca II line) by doing:

In [2]:
data = rh15d.Rh15dout('output/')
data.close()
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

--- Read output/output_aux.hdf5 file.
--- Read output/output_indata.hdf5 file.
--- Read output/output_ray.hdf5 file.


/uio/hume/student-u09/meudnaes/.local/lib/python3.7/site-packages/helita-0.9.0-py3.7-linux-x86_64.egg/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/uio/hume/student-u09/meudnaes/.local/lib/python3.7/site-packages/helita-0.9.0-py3.7-linux-x86_64.egg/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray = xr.open_dataset(infile, autoclose=True)


### Investigating the line

We plot the intensity over the relevant wavelengths.

In [3]:
intensity = data.ray.intensity
fig, ax = plt.subplots()
ax.plot(wave[indices], intensity[0,0,indices])
ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Intensity [W/(Hz nm2 sr)]")
ax.set_title("Ca II line");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Detailed output

To get a more detailed output, we need to run RH again over with the indices for the spectral line. We write these to the file `ray.input`. We also obtain the index for $\lambda = 500 \, \textrm{nm}$, and write it to this file. We get its index with:

In [4]:
wave.sel(wavelength=500, method='nearest')
index500 = np.argmin(np.abs(wave.data - 500))

To save wavelength indices into the file `ray.input` we do:

In [5]:
f = open('ray.input', 'w')  # this will overwrite any existing file!
f.write('1.00\n')
output = str(len(indices) + 1)
for ind in indices:
    output += ' %i' % ind
output += ' %i\n' % index500 
f.write(output)
f.close()

Now we run the RH code again. NB! Might have to shut down the notebook kernel for RH to be able to access input files. At least close all files RH needs to use (e.g. `ray.input`).

### Calculating optical depths

We can obtain the optical depths for $500\,\textrm{nm}$ and the line core by integrating $\chi$ (chi) over height.

In [6]:
from scipy.integrate import cumtrapz

height = data.atmos.height_scale[0, 0].dropna('height')  # first column
index500 = np.argmin(np.abs(data.ray.wavelength_selected - 500))  # index of 500 nm
tau500 = cumtrapz(data.ray.chi[0, 0, :, index500].dropna('height'), x=-height)
tau500 = np.concatenate([[1e-20], tau500])  # ensure tau500 and height have same size

index_core = np.argmin(np.abs(data.ray.wavelength_selected - 393.37))
tau_core = cumtrapz(data.ray.chi[0, 0, :, index_core].dropna('height'), x=-height)
tau_core = np.concatenate([[1e-20], tau_core])

We plot $\tau_{500}$ and $\tau_\textrm{core}$ vs height, and visualise where optical depth reaches unity:

In [7]:
fig, ax = plt.subplots()
ax.plot(height / 1e6, tau500, label=r"$\tau_{500}$")  # height in Mm
ax.plot(height / 1e6, tau_core, label=r"$\tau_{core}$")
ax.set_xlabel("Height (Mm)")
ax.set_ylabel(r"$\tau_\lambda$")
ax.set_title("Optical Depth")
ax.set_yscale("log")
ax.axhline(y=1, color='k', ls='--', label=r"$\tau=1$")
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

- At what height does $\tau$ reach unity at 500 nm?

In [8]:
tau_unity = height[np.argmin(np.abs(tau500 - 1))].values/1e6
display(HTML(r"$\tau_{500}$ reaches unity at %.4f Mm" %tau_unity)) 

- What about in the core of your Ca II line?

In [16]:
tau_unity = height[np.argmin(np.abs(tau_core - 1))].values/1e6
display(HTML(r"$\tau_\textrm{core}$ reaches unity at %.2f Mm" %tau_unity)) 

- Plot the departure coefficients for the ground Ca II level on a height scale and on a $\tau_{500}$ scale

We find the departure coefficients by dividing the level populations for $\textrm{Ca II}$ with its own LTE populations.

First we plot against a height scale.

In [24]:
pops = data.atom_CA.populations[0, 0, 0].dropna('height')
lte_pops = data.atom_CA.populations_LTE[0, 0, 0].dropna('height')

plt.figure()
plt.semilogy(height, pops/lte_pops, label='Ground state')
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Then we plot the departure coefficient against $\tau_{500}$

In [20]:
plt.figure()
plt.loglog(tau500, pops/lte_pops, label='Ground state')
plt.legend(loc="upper left")
plt.xlabel(r"$\tau_{500}$")
plt.ylabel("Departure coefficients")
plt.xlim(max(tau500), min(tau500));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We see that as we move upwards in the atmosphere, the departure coefficients go from being 1, to increasing to factors greater than $10^7$. This means that LTE is valid deep down in the atmosphere, while higher up in the atmosphere (around height $=1\,\textrm{Mm}$ or $\tau = 10^{-5}$) LTE breaks down.

- Plot the $\tau=1$ height as function of wavelength for the Ca II line

First we sum to find optical depth over all wavelength points.

In [12]:
waves = data.ray.wavelength_selected
index_line = np.arange(len(waves))[(waves > 392.8) & (waves < 394.0)]
y_arr = data.ray.chi[0, 0, :, index_line].dropna('height')
tau_line = cumtrapz(y_arr, x=-height, axis=0, initial=1e-20)

Then we find out at which height $\tau = 1$ for each wavelength and plot it

In [14]:
tau_unity = height[np.argmin(np.abs(tau_line - 1), axis=0)].values/1e6
fig, ax = plt.subplots()
ax.plot(waves[index_line], tau_unity)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'Height$(\tau = 1)$')
ax.set_title("Line formation");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

At $\tau = 1$ is basically the point in the atmosphere where the line forms. Since the line is most optically thick in the core, this part forms highest up in the atmosphere.

### Source Function

We use the SourceFunction widget from rh15d_vis

In [15]:
rh15d_vis.SourceFunction(data);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='wavelength', max=43), Checkbox(value=True, description='…

At the wings of the $\textrm{Ca II}$ line, the Source function departs from the Planck function at $0.6\,\textrm{Mm}$ (drag slider to the far left or right). At the line core ($\lambda = 393.366\,\textrm{nm} $ or index 21), the Source function departs from the Planck function at approximately the same height. The emissivity follows the Source much higher in the atmosphere in the line core.